In [3]:
#!pip install h3
import json
import branca.colormap as cm
from geojson import Feature, Point, FeatureCollection
from geopandas.tools import sjoin
from h3 import geo_to_h3, h3_to_geo_boundary
from shapely.geometry import Polygon
import seaborn as sns

#%%capture
import ast
import gc
import shapely.wkt
import shapely.geometry as geom
from shapely.ops import unary_union
from shapely.geometry import shape


from utils_spatial import *
# temp token
#access_token

## Parameters definition

In [208]:
countries = pd.read_excel(scldatalake + 'Manuals and Standards/IADB country and area codes for statistical use/IADB_country_codes_admin_0.xlsx', engine='openpyxl')
countries = np.unique(countries.isoalpha3)
#countries = countries[:3]
countries

array(['ABW', 'AIA', 'ARG', 'ATG', 'BES', 'BHS', 'BLM', 'BLZ', 'BOL',
       'BRA', 'BRB', 'BVT', 'CHL', 'COL', 'CRB', 'CRI', 'CUB', 'CUW',
       'CYM', 'DMA', 'DOM', 'ECU', 'FLK', 'GLP', 'GRD', 'GTM', 'GUF',
       'GUY', 'HND', 'HTI', 'JAM', 'KNA', 'LAC', 'LCA', 'MAF', 'MEX',
       'MSR', 'MTQ', 'NIC', 'OECD', 'PAN', 'PER', 'PRI', 'PRY', 'SGS',
       'SLV', 'SUR', 'SXM', 'TCA', 'TTO', 'URY', 'VCT', 'VEN', 'VGB',
       'VIR'], dtype=object)

In [4]:
# Country and Amenity
###########################
amenity = 'hospital'

# Transportation profile
###########################
profile='driving'
time_profiles=[45, 15, 30]

amenities_full = pd.read_csv(scldatalake + 'Geospatial infrastructure/Healthcare Facilities/healthcare_facilities.csv', low_memory=False)

In [13]:
#amenities_full = pd.read_csv(scldatalake + 'Geospatial infrastructure/Education Facilities/education_facilities.csv', low_memory=False)
#len(amenities_full)*3*4

In [14]:
#len(amenities_iso[amenities_iso.duplicated('lon')].sort_values('lon'))

## Get amenity from OSM

In [15]:
isochrones = []

In [411]:
%%capture
for isoalpha3 in countries:
    amenities_iso = amenities_full[amenities_full.isoalpha3 == isoalpha3].reset_index(drop=True).head(10)
    try:
        project_name = f'{isoalpha3}_{amenity}' 
        isochrones_data = create_isochrone_analysis(data = amenities_iso, project_name=project_name,
                                                    output_folder='../data', token=access_token, 
                                                    profile=profile, time_profiles=time_profiles)    
        isochrones_data['isoalpha3'] = isoalpha3
        isochrones.append(isochrones_data)  
        print('true ' + isoalpha3)
    except:
        print('Error ' + isoalpha3)

In [412]:
output = pd.concat(isochrones)
output['amenity'] = 'hospital'

In [414]:
old = pd.read_csv(scldatalake + 'Geospatial infrastructure/Healthcare Facilities/isochrones/OSM_hospital_LAC_isochrones_simplify.csv')
old = old[old.isoalpha3.isin(['GUY', 'CRI'])]
#old

In [415]:
new = pd.concat([old, output])
new = new[['isoalpha3', 'amenity', 'profile', 'minutes', 'multipolygon']]
new.to_csv(scldatalake + 'Geospatial infrastructure/Healthcare Facilities/isochrones/OSM_hospital_LAC_isochrones_simplify.csv', 
           index=False)

## Update

In [35]:
#isochrones_all = pd.read_csv(scldatalake + 'Geospatial infrastructure/Healthcare Facilities/isochrones/OSM_hospital_LAC_isochrones.csv')
#isochrones_all
#new = pd.concat([isochrones_all, output]).drop_duplicates()
#new.to_csv(scldatalake + 'Geospatial infrastructure/Healthcare Facilities/isochrones/OSM_hospital_LAC_isochrones.csv', index=False)
#new = output
#new = new[new.isoalpha3.isin(['GUY', 'ARG'])]
#new
#geom_ = gpd.GeoDataFrame.from_features(eval(new[(new.isoalpha3=='GUY') & (new.minutes==45)]['geometry'][0]))
# multi_polygon = to_multipolygon(new[(new.isoalpha3=='GUY') & (new.minutes==45)])
#import shapely.geometry as geom
#temp = new[['isoalpha3', 'geometry']][(new.isoalpha3=='GUY') & (new.minutes==45)]['geometry']
#geom_ = gpd.GeoDataFrame.from_features(eval(temp.values[0]))
#multi_polygon = geom.MultiPolygon(geom_['geometry'].tolist())

In [2]:
col_list = ['isoalpha3', 'amenity', 'profile', 'minutes', 'geometry']
new = pd.read_csv(scldatalake + 'Geospatial infrastructure/Healthcare Facilities/isochrones/OSM_hospital_LAC_isochrones.csv',
                  #usecols=col_list,
                  nrows=2
                 )
#new = pd.read_csv('../../../../OSM_hospital_LAC_isochrones.csv', usecols=col_list)
#new = new[['isoalpha3', 'amenity', 'profile', 'minutes', 'geometry']]

In [9]:
def to_multipolygon(row):
    try:
        geom_ = gpd.GeoDataFrame.from_features(ast.literal_eval(row['geometry']))
        # geom.MultiPolygon(geom_['geometry'].tolist())
        return unary_union(geom_['geometry'].tolist())
    except:
        return None

#new['multipolygon'] = new.apply(lambda x: to_multipolygon(x), axis=1)
new['multipolygon'] = None
for each in range(len(new)):
    if (new.iloc[each]['multipolygon'] == None) & (new.iloc[each]['isoalpha3'] not in ['BRA', 'ARG', 'MEX', 'PER']):
        new.loc[each, 'multipolygon'] = to_multipolygon(new.iloc[each])
        #new.loc[each, 'geometry'] = None
        gc.collect()
        print(new.iloc[each]['isoalpha3'])

In [19]:

#new.head(5)
row = new.iloc[0]
#geom_ = gpd.GeoDataFrame.from_features(ast.literal_eval(row['geometry']))
#tt = ast.literal_eval(row['geometry'])
#tt = json.loads(row['geometry'][1:-1])
#row['geometry'][1:-1]
#tt = ast.literal_eval()
#row['geometry'][1:-1].split('}, {')[1]
#geom_pandas = pd.DataFrame(tt)
#temp=shape(geom_pandas.geometry)
#test = unary_union(geom_['geometry'].tolist())

#p = gpd.GeoSeries(test)
#p.plot()
#plt.show()

#new.isoalpha3

In [25]:
geom_ = pd.DataFrame(ast.literal_eval(row.geometry))
from shapely.geometry import Polygon
from geojson import Polygon
geom_ = gpd.GeoDataFrame(geom_.geometry)
geom_

,geometry
0,"{'coordinates': [[[-70.049194, 12.623888], [-7..."
1,"{'coordinates': [[[-70.044925, 12.618621], [-7..."
2,"{'coordinates': [[[-70.046602, 12.623212], [-7..."


In [ ]:
new.iloc[2]['isoalpha3']

In [5]:
new = new[['isoalpha3', 'amenity', 'profile', 'minutes', 'multipolygon']]
new.to_csv(scldatalake + 'Geospatial infrastructure/Healthcare Facilities/isochrones/OSM_hospital_LAC_isochrones_simplify.csv', index=False)

In [ ]:
p = gpd.GeoSeries(new.multipolygon[10])
p.plot()
plt.show()